# Sprint アンサンブル学習

In [1]:
import numpy as np
import random
import pandas as pd
from numpy import linalg as LA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # testとtrain分割
from sklearn.preprocessing import StandardScaler #　標準化モジュール
from sklearn.linear_model import LinearRegression # 線形回帰
from sklearn.metrics import mean_squared_error # 平均二乗誤差
from sklearn.tree import DecisionTreeRegressor # 決定木
from sklearn.neighbors import KNeighborsClassifier # 最近傍法
from sklearn.svm import SVR # こっちはSVM回帰　最適化すると強いが手間がかかる、重い
from sklearn.svm import SVC # SVM
from sklearn.preprocessing import MinMaxScaler # データが0〜1の間に収まるよう変換
from sklearn.model_selection import GridSearchCV

In [2]:
# データセットの用意

In [3]:
Ames = pd.read_csv("train.csv")
Ames.head(4)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000


In [4]:
# 欠損値の確認

# 欠損値の有無を確認する
nan_Ames = pd.DataFrame(Ames.isnull().sum()) # 列ごとの欠損値の合計。
nan_Ames.columns = ["nan"]
nan_Ames.head()

,nan
Id,0
MSSubClass,0
MSZoning,0
LotFrontage,259
LotArea,0


In [5]:
# GrLivAreaとYearBuiltと目的変数SalePriceを抜き出す
Ames = Ames.loc[:, ["GrLivArea", "YearBuilt", "SalePrice"]]
Ames = Ames.astype(float)
Ames.head()

,GrLivArea,YearBuilt,SalePrice
0,1710.0,2003.0,208500.0
1,1262.0,1976.0,181500.0
2,1786.0,2001.0,223500.0
3,1717.0,1915.0,140000.0
4,2198.0,2000.0,250000.0


In [6]:
# 訓練用（train）データ８０%、検証用（validation）データ２０%に分割する

# 特徴量データから"GrLivArea", "YearBuilt"を抜き出しXに格納
X = Ames.loc[:, ["GrLivArea", "YearBuilt"]]
X = np.array(X)
X = X.astype(float)
print("X\n{}".format(X.shape))

# 目的変数SalePrice列を抜き出し、yのndarrayに格納
# 特徴量をX,目的変数をyのndarrayに格納
y = Ames.loc[:, ["SalePrice"]]
y = np.array(y)
#print("y_array\n{}".format(y_array[:4]))
print(y.shape)

y = y.ravel()# 1次元配列に変換
y = y.astype(float)
print(y.dtype)

#print("１次元配列に変換したy_array\n{}".format(y_array[:4]))


# データを訓練用と検証用に分割 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, test_size=0.2, random_state=0)

# Xの標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print("X_train")
print(X_train.shape)
print(X_train[:4])
print("y_train")
print(y_train.shape)
print(y_train[:4])
print("X_test")
print(X_test.shape)
print(X_test[:4])
print("y_test") 
print(y_test.shape)
print(y_test[:4])

X
(1460, 2)
(1460, 1)
float64
X_train
(1168, 2)
[[ 0.60188649  1.18803167]
 [-1.21671763 -0.29250097]
 [-1.08041967 -1.64143071]
 [ 0.3312377   1.02352804]]
y_train
(1168,)
[314813. 109500. 163500. 271000.]
X_test
(292, 2)
[[ 1.93955355 -0.4570046 ]
 [ 0.11510809 -0.9834162 ]
 [-0.61505952 -0.68730967]
 [ 0.97962654  0.20100991]]
y_test
(292,)
[200624. 133000. 110000. 192000.]


### 【問題1】ブレンディングのスクラッチ実装

In [7]:
# 単一モデル　：線形回帰
reg = LinearRegression().fit(X_train, y_train) # 学習
one_pre = reg.predict(X_test) # 推定

print("正解：y_test\n{}".format(y_test[:4]))
print("単一モデル(線形回帰)：推定\n{}".format(one_pre[:4]))

# 平均二乗誤差（MSE）
mse = mean_squared_error(y_test, one_pre)
print("MSE:\n{}".format(mse))

正解：y_test
[200624. 133000. 110000. 192000.]
単一モデル(線形回帰)：推定
[264908.90812295 155745.99630863 127984.53226316 236857.47974938]
MSE:
2942066921.672107


#### モデル１（線形回帰／SVM／決定木のブレンド）

In [8]:
#モデル１
#線形回帰
#SVM
#決定木

# 線形回帰による学習、推定
m1_reg = LinearRegression().fit(X_train, y_train) # 学習
m1_reg_pred = m1_reg.predict(X_test) # 推定
#print("線形回帰（単一モデル）：推定\n{}".format(m1_reg_pred[:4]))
# 平均二乗誤差（MSE）
mse = mean_squared_error(y_test, m1_reg_pred)
print("線形回帰（単一モデル）MSE:\n{}".format(mse))

# SVMによる学習、推定
m1_svr = SVR(gamma='scale', C=1.0, epsilon=0.2)
m1_svr.fit(X_train, y_train) # 学習
m1_svr_pred = m1_svr.predict(X_test) # 推定
#print("SVM（単一モデル）:推定\n{}".format(m1_svr_pred[:4]))
# 平均二乗誤差（MSE）
mse = mean_squared_error(y_test, m1_svr_pred)
print("SVM（単一モデル）MSE:\n{}".format(mse))

# 決定木による学習、推定
m1_tree = DecisionTreeRegressor(max_depth = 3)
m1_tree.fit(X_train, y_train)
m1_tree_pred = m1_tree.predict(X_test)
#print("決定木（単一モデル）:推定\n{}".format(m1_tree_pred[:4]))
# 平均二乗誤差（MSE）
mse = mean_squared_error(y_test, m1_tree_pred)
print("決定木（単一モデル）MSE:\n{}".format(mse))


線形回帰（単一モデル）MSE:
2942066921.672107
SVM（単一モデル）MSE:
7221625115.268777
決定木（単一モデル）MSE:
2695315830.674088


In [16]:
# 推定方法の精度を踏まえ、重み付け（線形回帰０.２、SVM０.６、決定木０.２）

m1_reg_weight = np.array(m1_reg_pred*0.1)
m1_svr_weight = np.array(m1_svr_pred*0.1)
m1_tree_weight = np.array(m1_tree_pred*0.8)

m1 = m1_reg_weight +  m1_svr_weight + m1_tree_weight

# 平均二乗誤差（MSE）
mse = mean_squared_error(y_test, one_pre)
print("単一モデル（線形回帰）のMSE:\n{}".format(mse))
mse = mean_squared_error(y_test, m1)
print("モデル１(線形回帰/SVM/決定木のブレンド)のMSE:\n{}".format(mse))


単一モデル（線形回帰）のMSE:
2942066921.672107
モデル１(線形回帰/SVM/決定木のブレンド)のMSE:
2706205467.478844


#### ⬆︎モデル１(線形回帰/SVM/決定木のブレンド)で線形回帰の単一モデルより精度が上がった

### モデル２

In [10]:
# モデル２
#　標準化、
# グリッドリサーチしたSVM
# 

# Xの標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
print(X_train[:4, :])
print(X_test[:4, :])

[[ 0.60188649  1.18803167]
 [-1.21671763 -0.29250097]
 [-1.08041967 -1.64143071]
 [ 0.3312377   1.02352804]]
[[ 1.93955355 -0.4570046 ]
 [ 0.11510809 -0.9834162 ]
 [-0.61505952 -0.68730967]
 [ 0.97962654  0.20100991]]


In [11]:
# モデル３



In [12]:
X_train = np.zeros((1, X.shape[1]))
print(X_train)

[[0. 0.]]


### 【問題2】バギングのスクラッチ実装

In [13]:
# N種類のサブセットを作成、線形回帰で学習と推定の関数
class ScratchBagging():
        
    # XデータをN回抽出して結合する関数(Xのshape)
    def Bootstrap(self, X, y, N):
        self.X_0 = np.empty((0, X.shape[1])) # 空の配列を用意
        self.y_0 = np.empty((0, )) # 空の配列を用意
        for i in range(N+1): # Nでサンプル数が割り切れないことがあるので多めに作る
            X_0, X_, y_0, y_ = train_test_split(
                X, y, train_size=(1/N), test_size=(1-(1/N)), shuffle=True, random_state=None)
            self.X_0 = np.array(X_0)
            self.y_0 = np.array(y_0)
            self.X_0 = np.append(self.X_0, X_0, axis=0)
            self.y_0 = np.append(self.y_0, y_0, axis=0)
        
        self.X_0 = self.X_0[:X.shape[0], :] # Xの行数以上の行は削除
        self.y_0 = self.y_0[:y.shape[0]]

        #print(self.X_0.shape)
        return self.X_0, self.y_0
        
    def fit_predict(self, model, X, y, N):
        
        # 固定用ブートストラップサンプル１個作る(固定のX_test作るため)
        X_fix, y_fix = self.Bootstrap(X, y, N)
        
        #20%のX_testを取り出す
        X_train_fix, X_test_fix, y_, self.y_test_fix= train_test_split(
                X_fix, y_fix, train_size=0.8, test_size=0.2, shuffle=True, random_state=0)
        # X_testの標準化
        scaler = StandardScaler()
        scaler.fit(X_train_fix)
        X_test_fix = scaler.transform(X_test_fix)#yじゃないからやってよし？
        
        # 指定したmodelでN回推定
        self.N_pred = [] # N回分の推定値を入れるリスト
        for i in range(N):
            X_n, y_n = self.Bootstrap(X, y, N) # ブートストラップサンプル１個作る(Xのshape)
            
            # X,yのtrainデータを取り出す
            X_train, X_, y_train, y_ = train_test_split(
                X_n, y_n, train_size=0.8, test_size=0.2, shuffle=True, random_state=None)
            X_train = np.array(X_train)
            y_train = np.array(y_train)

            # X_trainの標準化
            scaler.fit(X_train)
            X_train = scaler.transform(X_train) 

            # yを1次元に直す（modelに入れた時エラー出ないように）
            y_train = np.ravel(y_train)
            
            #指定のモデルで学習する(インスタンス化)
            fit_model = model.fit(X_train, y_train)
            
            #推定
            fit_model_pred = fit_model.predict(X_test_fix)
            self.N_pred.append(fit_model_pred) # 推定結果をリストに追加していく
        
        #N個の推定値をnumpyに直す
        self.N_pred = np.array(self.N_pred)
        #print(self.N_pred.shape)
        
        #推定値の平均を返す
        self.N_pred = np.mean(self.N_pred, axis=0)

        return self.N_pred


In [14]:
#モデルのメモ

# SVM
# 

# ScratchBagging(線形回帰: LinearRegression()) で推定
n4 = ScratchBagging()
n4_pred = n4.fit_predict(LinearRegression(), X, y, 4)
n4_y_test = n4.y_test_fix
mse = mean_squared_error(n4_y_test, n4_pred)
print("\nバギングした線形回帰のMSE:\n{}".format(mse))
mse = mean_squared_error(y_test, one_pre)
print("線形回帰のMSE:\n{}".format(mse))

# ScratchBagging(SVM：SVR(gamma='scale', C=1.0, epsilon=0.2)) で推定
n4_pred = n4.fit_predict(SVR(gamma='scale', C=1.0, epsilon=0.2), X, y, 4)
n4_y_test = n4.y_test_fix
mse = mean_squared_error(n4_y_test, n4_pred)
print("\nバギングしたSVMのMSE:\n{}".format(mse))
mse = mean_squared_error(y_test, one_pre)
print("SVMのMSE:\n7235023974.812659")

# ScratchBagging(決定木：DecisionTreeRegressor(max_depth = 3)) で推定
n4_pred = n4.fit_predict(DecisionTreeRegressor(max_depth = 3), X, y, 4)
n4_y_test = n4.y_test_fix
mse = mean_squared_error(n4_y_test, n4_pred)
print("\nバギングした決定木のMSE:\n{}".format(mse))
print("決定木のMSE:\n2695315830.674088")


バギングした線形回帰のMSE:
1887472131.870954
線形回帰のMSE:
2942066921.672107

バギングしたSVMのMSE:
5319702684.948857
SVMのMSE:
7235023974.812659

バギングした決定木のMSE:
1224625919.9329352
決定木のMSE:
2695315830.674088


### ⬆︎線形回帰、SVMはバギングで単一モデルより精度が上がった⬆︎

### 【問題3】スタッキングのスクラッチ実装

In [15]:
# ステージ０

# N種類のサブセットを作成、線形回帰で学習と推定の関数
class ScratchStacking():
    
    # ステージ０
    
    # 以下をMモデル分繰り返し、M個のブレンドデータを得る
    
        # 以下をK回繰り返してインスタンスをK個作る（ブレンドデータ）
            # Xをtrainとvalidationに分割
            # モデル１でtrainで学習、validationで推定する
     
    # ステージ１
    
    # ステージ０のブレンドデータを
    
    def make_X_train(self, X, N):
        X_train = np.empty((0, X.shape[1]))　# 0
        for i in range(N+1): # Nでサンプル数が割り切れないことがあるので多めに作る
            #X_train, X_test = train_test_split(X, test_size=(1/N), train_size=(1-(1/N)), shuffle=True, random_state=None)
            #X_t = np.array(X_t) # shape(365.2)
            #X_test = np.append(X_test, X_t, axis=0)
            #X_train をまとめたデータ作る

        #print(X_train.shape)
        X_train = X_train[:X.shape[0], :] # Xのサンプル数以上の行は削除
        #print(X_train.shape)
        return X_train, X_test
        
    def brend(self, model1, model2, model3, X, y, N):
        
        #train_testに分割
        X_train, X_test = train_test_split(
            X, test_size=(1/N), train_size=(1-(1/N)), shuffle=True, random_state=None)
                
        y = np.ravel(y)# SVMでエラーが出るので、１次元配列にする                
        self.N_pred = [] # N回分の推定値を入れるリスト
        
        # 指定したmodel１、２、３で推定
        model_list = [model1, model2, model3]
        for i in range(len(model_list)):
            
            model[i].fit(X_train, y) # インスタンス化、学習
            self.N_pred.append(model[i].predict(X_train)) # 推定結果をリストに追加していく
        
        self.N_pred = np.array(self.N_pred)
        #print(self.N_pred.shape)
        
        #推定値の平均を返す
        self.N_pred = np.mean(self.N_pred, axis=0)

        return self.N_pred


SyntaxError: invalid character in identifier (<ipython-input-15-eed56a323cf3>, line 19)

In [ ]:
X_train = np.empty((0, X.shape[1]))
print(X_train)
print(X_train.shape)